# Bài tập SQL chương 3

In [39]:
import sqlite3
import pandas as pd
import numpy as np  
import math


## Thiết lập kết nối Database

Tạo một database SQLite (nếu chưa có) và thiết lập kết nối.

In [40]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
conn.create_function("SQRT", 1, math.sqrt)

---
## Câu 1: Viết câu lệnh SQL để tính sự tuong quan giữa A và B

Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức Pearson Correlation Coefficient:

Công thức: `r_AB = [n * Σ(a_i * b_i) - (Σa_i * Σb_i)] / sqrt([n * Σa_i² - (Σa_i)²] * [n * Σb_i² - (Σb_i)²])`

**Giả định:** Có một bảng `DataPoints` chứa các cặp giá trị (value_a, value_b).


In [41]:
#Tạo bảng
cursor.execute("""
CREATE TABLE DataPoints (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    value_a REAL,
    value_b REAL
);
""")
# Dữ liệu mẫu đơn giản
sample_data = [
    (2, 1),
    (4, 2),
    (5, 4),
    (6, 4),
    (8, 5)
]
cursor.executemany("INSERT INTO DataPoints (value_a, value_b) VALUES (?, ?);", sample_data)
conn.commit()

In [42]:
# 2. Viết truy vấn SQL tính r_AB

query_c1 = """
WITH Aggregates AS (
    SELECT
        COUNT(*) as n,
        SUM(value_a) as sum_a,
        SUM(value_b) as sum_b,
        SUM(value_a * value_b) as sum_ab,
        SUM(value_a * value_a) as sum_a_sq, -- SUM(POWER(value_a, 2)) không dùng được trong SQLite
        SUM(value_b * value_b) as sum_b_sq  -- SUM(POWER(value_b, 2)) không dùng được trong SQLite
    FROM
        DataPoints
)
SELECT
    agg.n,
    agg.sum_a,
    agg.sum_b,
    agg.sum_ab,
    agg.sum_a_sq,
    agg.sum_b_sq,
    -- Tính toán tử số
    (agg.n * agg.sum_ab - agg.sum_a * agg.sum_b) AS numerator,
    -- Tính toán các thành phần mẫu số
    (agg.n * agg.sum_a_sq - agg.sum_a * agg.sum_a) AS denominator_a_term,
    (agg.n * agg.sum_b_sq - agg.sum_b * agg.sum_b) AS denominator_b_term,
    -- Tính toán mẫu số (tránh lỗi chia cho 0 hoặc căn bậc hai của số âm)
    CASE
        WHEN (agg.n * agg.sum_a_sq - agg.sum_a * agg.sum_a) <= 0 OR (agg.n * agg.sum_b_sq - agg.sum_b * agg.sum_b) <= 0 THEN NULL -- Trả về NULL nếu phương sai bằng 0 hoặc âm (không hợp lệ)
        ELSE SQRT((agg.n * agg.sum_a_sq - agg.sum_a * agg.sum_a) * (agg.n * agg.sum_b_sq - agg.sum_b * agg.sum_b))
    END AS denominator,
    -- Tính r_AB (hệ số tương quan)
    CASE
        WHEN (agg.n * agg.sum_a_sq - agg.sum_a * agg.sum_a) <= 0 OR (agg.n * agg.sum_b_sq - agg.sum_b * agg.sum_b) <= 0 THEN NULL -- Trả về NULL nếu mẫu số không hợp lệ
        -- Sử dụng NULLIF để tránh chia cho 0 nếu mẫu số tính ra chính xác bằng 0
        ELSE (agg.n * agg.sum_ab - agg.sum_a * agg.sum_b) / NULLIF(SQRT((agg.n * agg.sum_a_sq - agg.sum_a * agg.sum_a) * (agg.n * agg.sum_b_sq - agg.sum_b * agg.sum_b)), 0)
    END AS pearson_correlation_rAB
FROM
    Aggregates agg;
"""

print("\n--- Kết quả Câu 1 (Tính r_AB) ---")
df_c1 = pd.read_sql_query(query_c1, conn)
print(df_c1)
print(f"\nHệ số tương quan Pearson (r_AB): {df_c1['pearson_correlation_rAB'].iloc[0]}")




--- Kết quả Câu 1 (Tính r_AB) ---
   n  sum_a  sum_b  sum_ab  sum_a_sq  sum_b_sq  numerator  denominator_a_term  \
0  5   25.0   16.0    94.0     145.0      62.0       70.0               100.0   

   denominator_b_term  denominator  pearson_correlation_rAB  
0                54.0    73.484692                 0.952579  

Hệ số tương quan Pearson (r_AB): 0.9525793444156803


---
## Câu 2: Chuyển đổi dữ liệu và chuẩn bị cho kiểm tra χ²

Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2.

|        | A   | B   | C   |
| :----- | :-: | :-: | :-: |
| Day 1  | 8   | 9   | 7   |
| Day 2  | 7.5 | 8.5 | 7   |
| Day 3  | 6   | 7   | 8   |
| Day 4  | 7   | 6   | 5   |


In [43]:
# Tạo bảng 
cursor.execute("""
CREATE TABLE CarTests (
    TestID INTEGER PRIMARY KEY AUTOINCREMENT,
    Day INTEGER,
    Model TEXT,
    Score REAL
);
""")

# Dữ liệu từ bảng đề bài
test_data = [
    (1, 'A', 8.0), (1, 'B', 9.0), (1, 'C', 7.0),
    (2, 'A', 7.5), (2, 'B', 8.5), (2, 'C', 7.0),
    (3, 'A', 6.0), (3, 'B', 7.0), (3, 'C', 8.0),
    (4, 'A', 7.0), (4, 'B', 6.0), (4, 'C', 5.0)
]

# Chèn dữ liệu vào bảng CarTests
cursor.executemany("INSERT INTO CarTests (Day, Model, Score) VALUES (?, ?, ?);", test_data)
conn.commit()

In [44]:
# 2. Hiển thị dữ liệu đã chuyển đổi

query_c2_show = "SELECT * FROM CarTests ORDER BY Day, Model;"

print("\n--- Kết quả Câu 2 (Dữ liệu dạng quan hệ) ---")
df_c2 = pd.read_sql_query(query_c2_show, conn)
print(df_c2)
print("\nDữ liệu đã sẵn sàng ở định dạng quan hệ.")
print("Để thực hiện kiểm tra Chi-Squared (χ²), cần xử lý thêm (ví dụ: phân loại điểm số)")
print("và sử dụng các thư viện thống kê bên ngoài SQL (như Scipy trong Python).")



--- Kết quả Câu 2 (Dữ liệu dạng quan hệ) ---
    TestID  Day Model  Score
0        1    1     A    8.0
1        2    1     B    9.0
2        3    1     C    7.0
3        4    2     A    7.5
4        5    2     B    8.5
5        6    2     C    7.0
6        7    3     A    6.0
7        8    3     B    7.0
8        9    3     C    8.0
9       10    4     A    7.0
10      11    4     B    6.0
11      12    4     C    5.0

Dữ liệu đã sẵn sàng ở định dạng quan hệ.
Để thực hiện kiểm tra Chi-Squared (χ²), cần xử lý thêm (ví dụ: phân loại điểm số)
và sử dụng các thư viện thống kê bên ngoài SQL (như Scipy trong Python).


---
## Câu 3: Chuyển đổi thời gian dạng số nguyên sang định dạng thời gian

Bảng `flights(departure_time_int, ...)` chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian `HH:MM`.

In [45]:
# 1. Tạo bảng và chèn dữ liệu mẫu
cursor.execute("""
CREATE TABLE flights (
    flight_id INTEGER PRIMARY KEY AUTOINCREMENT,
    departure_time_int INTEGER
);
""")

sample_flights = [
    (830,), (905,), (1445,), (1700,), (2359,), (5,), (1200,)
]
cursor.executemany("INSERT INTO flights (departure_time_int) VALUES (?);", sample_flights)
conn.commit()
print("Đã tạo bảng 'flights' và chèn dữ liệu mẫu.")

Đã tạo bảng 'flights' và chèn dữ liệu mẫu.


In [46]:
# 2. Thực hiện truy vấn chuyển đổi
query_c3 = """
SELECT
    flight_id,
    departure_time_int,
    printf('%02d', departure_time_int / 100) || ':' || printf('%02d', departure_time_int % 100) AS formatted_departure_time
FROM
    flights;
"""

print("\n--- Kết quả Câu 3 ---")
df_c3 = pd.read_sql_query(query_c3, conn)
print(df_c3)


--- Kết quả Câu 3 ---
   flight_id  departure_time_int formatted_departure_time
0          1                 830                    08:30
1          2                 905                    09:05
2          3                1445                    14:45
3          4                1700                    17:00
4          5                2359                    23:59
5          6                   5                    00:05
6          7                1200                    12:00


---
## Câu 4: Tìm ngoại lệ sử dụng MAD

Viết truy vấn SQL để tìm các giá trị ngoại lệ trong bảng `measurements(value)` bằng cách sử dụng MAD (Median Absolute Deviation). Một quy tắc chung là xem xét các giá trị ngoại lệ nếu độ lệch tuyệt đối của chúng so với median lớn hơn 1.5 lần MAD.


In [47]:
# 1. Tạo bảng và chèn dữ liệu mẫu
cursor.execute("""
CREATE TABLE measurements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    value REAL
);
""")
sample_measurements = [
    (10.5,), (11.2,), (10.8,), (11.5,), (10.9,), (11.0,),
    (11.1,), (10.7,), (12.0,), (25.0,), (1.0,) # 25.0 và 1.0 có thể là ngoại lệ
]
cursor.executemany("INSERT INTO measurements (value) VALUES (?);", sample_measurements)
conn.commit()
print("Đã tạo bảng 'measurements' và chèn dữ liệu mẫu.")

Đã tạo bảng 'measurements' và chèn dữ liệu mẫu.


In [48]:
# 2. Thực hiện truy vấn tìm ngoại lệ bằng MAD

query_c4 = """
WITH MedianCalc AS (
    -- Tính số thứ tự và tổng số hàng để xác định vị trí giữa
    SELECT
        id,
        value,
        ROW_NUMBER() OVER (ORDER BY value) as rn,
        COUNT(*) OVER () as total_count
    FROM measurements
), OverallMedian AS (
    -- Tính Median bằng cách lấy trung bình của giá trị ở vị trí giữa (hoặc hai vị trí giữa nếu số hàng chẵn)
    SELECT AVG(value) as median_val
    FROM MedianCalc
    WHERE rn IN ( (total_count + 1) / 2, (total_count + 2) / 2 )
), AbsoluteDeviations AS (
    -- Tính độ lệch tuyệt đối so với Median cho mỗi điểm dữ liệu
    SELECT
        m.id,
        m.value,
        om.median_val,
        ABS(m.value - om.median_val) as abs_dev
    FROM measurements m, OverallMedian om
), MADCalc AS (
    -- Tính số thứ tự và tổng số hàng cho các độ lệch tuyệt đối
    SELECT
        id,
        value,
        median_val,
        abs_dev,
        ROW_NUMBER() OVER (ORDER BY abs_dev) as rn_mad,
        COUNT(*) OVER () as total_count_mad
    FROM AbsoluteDeviations
), OverallMAD AS (
    -- Tính MAD (Median của các độ lệch tuyệt đối)
    -- Xử lý trường hợp MAD = 0 để tránh chia cho 0 nếu tất cả các giá trị giống hệt nhau
    SELECT MAX(mad_val, 1e-9) as mad_val -- Sử dụng MAX để tránh MAD = 0
    FROM (
        SELECT AVG(abs_dev) as mad_val
        FROM MADCalc
        WHERE rn_mad IN ( (total_count_mad + 1) / 2, (total_count_mad + 2) / 2 )
    )
)
-- Cuối cùng, chọn ra các điểm dữ liệu là ngoại lệ
SELECT
    ad.id,
    ad.value,
    ad.median_val,
    ad.abs_dev,
    oM.mad_val,
    (ad.abs_dev / oM.mad_val) AS mad_score -- Điểm số MAD (tùy chọn)
FROM
    AbsoluteDeviations ad, OverallMAD oM
WHERE
    ad.abs_dev > (1.5 * oM.mad_val); -- Điều kiện ngoại lệ
"""

print("\n--- Kết quả Câu 4 (Ngoại lệ dùng MAD) ---")
df_c4 = pd.read_sql_query(query_c4, conn)
print(df_c4)


--- Kết quả Câu 4 (Ngoại lệ dùng MAD) ---
   id  value  median_val  abs_dev  mad_val  mad_score
0   1   10.5        11.0      0.5      0.3   1.666667
1   4   11.5        11.0      0.5      0.3   1.666667
2   9   12.0        11.0      1.0      0.3   3.333333
3  10   25.0        11.0     14.0      0.3  46.666667
4  11    1.0        11.0     10.0      0.3  33.333333


---
## Câu 5: Xác định bệnh nhân trùng lặp bằng khoảng cách Boolean kết hợp

Xác định liệu hai người trong bảng `Patient(patient_id, last_name, weight, height)` có phải là một người hay không bằng cách sử dụng "khoảng cách kết hợp Boolean" trên `last_name` và `weight`. Nghĩa là, tìm các cặp bệnh nhân có cùng `last_name` VÀ cùng `weight`.

In [49]:
# 1. Tạo bảng và chèn dữ liệu mẫu
cursor.execute("""
CREATE TABLE Patient (
    patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
    last_name TEXT,
    weight REAL,
    height REAL
);
""")
sample_patients = [
    ('Nguyen', 65.5, 170.0),
    ('Tran', 70.0, 175.0),
    ('Le', 55.2, 160.5),
    ('Nguyen', 65.5, 171.0), # Trùng last_name và weight với ID 1
    ('Pham', 80.1, 180.0),
    ('Tran', 70.0, 175.5), # Trùng last_name và weight với ID 2
    ('Nguyen', 75.0, 172.0) # Khác weight với ID 1 và 4
]
cursor.executemany("INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?);", sample_patients)
conn.commit()
print("Đã tạo bảng 'Patient' và chèn dữ liệu mẫu.")

Đã tạo bảng 'Patient' và chèn dữ liệu mẫu.


In [50]:
# 2. Thực hiện truy vấn tìm cặp trùng lặp
query_c5 = """
SELECT
    p1.patient_id AS patient1_id,
    p1.last_name AS patient1_lastname,
    p1.weight AS patient1_weight,
    p1.height AS patient1_height,
    ' | ' AS separator, -- Phân tách giữa 2 bệnh nhân
    p2.patient_id AS patient2_id,
    p2.last_name AS patient2_lastname,
    p2.weight AS patient2_weight,
    p2.height AS patient2_height
FROM
    Patient p1
JOIN
    Patient p2 ON p1.patient_id < p2.patient_id -- Chỉ lấy mỗi cặp 1 lần và không so sánh với chính nó
WHERE
    p1.last_name = p2.last_name  -- Điều kiện 1: Cùng last_name
    AND p1.weight = p2.weight;   -- Điều kiện 2: Cùng weight
"""

print("\n--- Kết quả Câu 5 (Bệnh nhân trùng lặp tiềm năng) ---")
df_c5 = pd.read_sql_query(query_c5, conn)
print(df_c5)


--- Kết quả Câu 5 (Bệnh nhân trùng lặp tiềm năng) ---
   patient1_id patient1_lastname  patient1_weight  patient1_height separator  \
0            1            Nguyen             65.5            170.0        |    
1            2              Tran             70.0            175.0        |    

   patient2_id patient2_lastname  patient2_weight  patient2_height  
0            4            Nguyen             65.5            171.0  
1            6              Tran             70.0            175.5  


In [51]:
cursor.close()
conn.close()